In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import re
import os
import pickle

In [2]:
def read_from_pickle(directory, filename):
    df = pd.DataFrame()
    filepath = directory + filename
    with open(filepath, 'rb') as file:
        df = pickle.load(file)
            
    return df

In [3]:
directory = "/Users/dianaow/Documents/"
all_results = read_from_pickle(directory, "all_results.pickle")

## Data Scientist vs Not Data Scientist

In [4]:
test = all_results

In [5]:
def remove_lists(x):
    try:
        return x[0]     
    except:
        pass

In [6]:
df = test.applymap(remove_lists)

In [7]:
df.dropna(subset=['all info', 'company name', 'country', 'job title'], inplace=True)

In [8]:
df.isnull().sum()

all info        0
company name    0
country         0
job title       0
link            0
salary          0
dtype: int64

In [9]:
df.loc[:,"has salary"] = 0

df.loc[(df["all info"].str.contains("SGD") |
        df["all info"].str.contains("Salary")), "has salary"] = 1

In [12]:
# Check if job postings have salary written
def extract_salary(df):
    
    salaries = []
    for i,row in df.iterrows():
        if df.loc[i,'has salary'] == 1:
            try:
                salary = [x.strip() for x in re.findall('Salary(.+?)<br/>', str(df.loc[i,'all info']), re.MULTILINE | re.DOTALL)]
                salaries.append(salary)
            except:
                salaries.append("")
        else:
            salaries.append("")
                
    return salaries

In [13]:
salaries = extract_salary(df)

In [14]:
# Only 2 job postings have salary numbers 
salaries

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 [''],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 [''],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 [':\n4500\nSGD\nPer Month'],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 [],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ['range\nWe offer a competitive salary commensurate with experience, and benefits.'],
 '',
 '',
 [''],
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '

In [15]:
mapping = [ ('<li>', ''), ('</li>', ''), ('<p>', ''), ('</p>', ''), ('<strong>', ''), \
           ('</strong>', ''), ('<br>', ''), ('<br/>', ''), ('\n', ''), ('<b>', ''), \
           ('<ul>', ''), ('</ul>', ''), ('&amp', '')]
for k, v in mapping:
    df["all info"] = df["all info"].map(lambda x: str(x).replace(k, v))

In [16]:
df.shape

(4852, 7)

In [17]:
# Check for duplicate job postings
df['is_duplicated'] = df.duplicated(['all info', 'company name', 'job title'])
df['is_duplicated'].sum()

4195

In [18]:
df_nodup = df[df['is_duplicated'] == False]
df_nodup.shape

(657, 8)

In [19]:
df_nodup.loc[:,"has scientist"] = 0

df_nodup.loc[(df_nodup["job title"].str.contains("Scientist") |
        df_nodup["job title"].str.contains("Scientist")), "has scientist"] = 1

/Users/dianaow/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/dianaow/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
df_nodup.head(50)

,all info,company name,country,job title,link,salary,has salary,is_duplicated,has scientist
0,The purpose of the role is to provide advanced...,Options Group,Singapore,"<font size=""+1"">Data Scientist / Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,1
1,The big data engineer implements and maintains...,Siemens AG,Singapore,"<font size=""+1"">Big Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
2,"Incorporated in 1984, main board-listed Singap...",Singapore Press Holdings,North Singapore,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
3,You are responsible forManages the early invol...,Philips,Singapore,"<font size=""+1"">Supplier Quality Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
4,"In this intern program, you will be part of ou...",PropertyGuru,Singapore,"<font size=""+1"">Data engineer intern</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
5,You are interested in building robust data pla...,Vault Dragon,Toa Payoh,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
6,Do you have a passion for data? Are you lookin...,Moka,Singapore,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
7,Responsibilities:Conduct research and particip...,DSO National Laboratories,Singapore,"<font size=""+1"">Data Analytics Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
8,Healint is a leading maker of healthcare techn...,Healint,Pasir Panjang,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0
9,"Data Visualization Engineer,-174900Description...",Teradata,Singapore,"<font size=""+1"">Data Visualization Engineer,</...",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,0


In [21]:
X = df_nodup[["all info"]]
y = df_nodup["has scientist"]

In [22]:
y.value_counts()

0    594
1     63
Name: has scientist, dtype: int64

In [24]:
import nltk
nltk.download("stopwords")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dianaow/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2,stratify=y)

In [26]:
for row in X_train['all info']:
    print row
    print "-------------------"

PURPOSE OF THE ROLE</b>This role of a data analyst is to implement the customer analytics strategy for Aviva in all its Asia markets such as Singapore, Hong Kong, Vietnam, China, and IndonesiaACCOUNTABILITIES</b>Develop multi-dimensional campaign tracking reports/dashboards to be used by marketing, operations, and client managers.Analyze daily trends and provide actionable insights to internal stakeholders in optimizing business resultsBuild and maintain customer databases to be used for analytics, campaign tracking, financial modelling, and management reports.Assist with campaign design and execution by leveraging analytics to maximize performance and optimize future campaigns.Assist in designing and implementing website analytic (Adobe Analytics or similar) to track website traffic and conversionsAssist with development and implementation of statistical models (predictive models / machine learning) and other analytic toolsOUTCOMES</b>Provide good, accurate and quality analysis, predi

The candidate will participate in high level innovative research projects. An important task isto actively run clinical trials involving patients with ophthalmic conditions, exploring theirpupillary responses to light as well as eye movements. She or he will interact with scientists,physicians, research assistants, evaluating these parameters in various ophthalmic andneurologic conditions like glaucoma, diabetic retinopathy, dementia, etc, and using variousophthalmic devices, including pupillometry. The candidate will receive appropriatementoring support by the senior members of the team, with the aim to produce high impactfactor articles in ophthalmic and visual science.Minimum Entry Requirements</b>• PhD in visual science, neuroscience, optometry and related disciplines • Minimum of 3 years’ experience in clinical research (including years of mastersand/or PhD)• Excellent human contact, empathetic with patients • In depth data analysis, using SPSS, SigmaPlot or Matlab • Data interpre

-------------------
The Business System Development team helps ; supports the Global Manufacturing organisation by identifying and leading process improvements projects. As an Advanced Analytics Manager, you will be asked to propose, create and update innovative solutions to support the various initiatives.You will need to be proactive, innovative and professional and will be based in Singapore but may liaise with Malaysia / Philippines / UK and overseas offices.Market Overview</b>Our Singapore story began in 2004 and today we have more than 1000 people working here. At the beginning of 2017 we opened our new, state of the art, Singapore Technology Centre in Science Park I, minutes from Singapore’s technology and start-up community. This new Technology Centre houses some of Dyson’s most advanced facilities including Robotics and Sensors laboratories and Smart Home, Connectivity ; Mobile Applications laboratories as well as an Advanced Control Tower with the latest technology to develop

In [27]:
stop_words = stopwords.words('english')
stop_words.extend(['data', 'Data'])

In [30]:
def vectorizer_fitting(vec_type, df_train, df_test, stop_words, train=True, field="all info"):

    vec = vec_type(stop_words= stop_words, ngram_range=(1, 3), max_features=20)

    vec.fit(df_train[field])

    # Transform train set
    X_train = pd.DataFrame(vec.transform(df_train[field]).todense(),
                           columns=vec.get_feature_names())
    
    # Transform test set
    X_test  = pd.DataFrame(vec.transform(df_test[field]).todense(),
                 columns=vec.get_feature_names())

    if train == True:
        return X_train
    else:
        return X_test

In [31]:
X_train_new = vectorizer_fitting(TfidfVectorizer, X_train, X_test, stop_words, train=True, field="all info")
X_test_new = vectorizer_fitting(TfidfVectorizer, X_train, X_test, stop_words, train=False, field="all info")

In [32]:
X_train_new.head()

,ability,analysis,business,design,development,engineering,experience,knowledge,management,process,project,requirements,research,skills,support,team,technical,work,working,years
0,0.000000,0.232181,0.302497,0.136570,0.122945,0.000000,0.237176,0.114251,0.692831,0.000000,0.000000,0.1183,0.000000,0.482473,0.000000,0.000000,0.125391,0.000000,0.000000,0.094879
1,0.351305,0.083393,0.796754,0.098104,0.000000,0.090588,0.170374,0.000000,0.000000,0.000000,0.095185,0.0000,0.112695,0.000000,0.334466,0.000000,0.090074,0.066386,0.000000,0.204467
2,0.000000,0.000000,0.249981,0.000000,0.000000,0.000000,0.392002,0.000000,0.000000,0.000000,0.328508,0.0000,0.777878,0.000000,0.000000,0.000000,0.000000,0.000000,0.266143,0.000000
3,0.260489,0.123670,0.644492,0.000000,0.000000,0.134340,0.168441,0.000000,0.000000,0.138273,0.000000,0.0000,0.501374,0.102795,0.124001,0.293943,0.000000,0.098448,0.228720,0.101074
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [33]:
X_train_cvec = vectorizer_fitting(CountVectorizer, X_train, X_test, stop_words, train=True)

In [34]:
X_train_cvec.head()

,ability,analysis,business,design,development,engineering,experience,knowledge,management,process,project,requirements,research,skills,support,team,technical,work,working,years
0,0,2,3,1,1,0,3,1,6,0,0,1,0,5,0,0,1,0,0,1
1,4,1,11,1,0,1,3,0,0,0,1,0,1,0,4,0,1,1,0,3
2,0,0,2,0,0,0,4,0,0,0,2,0,4,0,0,0,0,0,2,0
3,2,1,6,0,0,1,2,0,0,1,0,0,3,1,1,3,0,1,2,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Classify job category using Logistic Regression

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [36]:
lr = LogisticRegression(random_state=123)
cross_val_score(lr,X_train_new,y_train)

array([ 0.9047619 ,  0.9047619 ,  0.90410959])

In [37]:
lr.fit(X_train_new,y_train)
lr.score(X_test_new,y_test)   # Accuracy on test set

0.90322580645161288

In [38]:
#target_names = ['data scientist', 'not data scientist']
print classification_report(y_test,lr.predict(X_test_new))

             precision    recall  f1-score   support

          0       0.90      1.00      0.95       196
          1       0.00      0.00      0.00        21

avg / total       0.82      0.90      0.86       217



/Users/dianaow/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Classify job category using Random Forest

In [39]:
# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV 

In [40]:
rfc_params = {
    'n_estimators':[5,10],
    'max_depth':[1,2,3,4,5,6,7],
    'max_features':[None,'log2','sqrt',2,5],
    #'min_samples_split':[2,3,4,5,10]
}


rfc_gs = GridSearchCV(RandomForestClassifier(random_state=6), \
                              rfc_params, cv=5, verbose=1,n_jobs=-1)#,scoring='f1')
rfc_gs.fit(X_train_new, y_train)

y_pred = rfc_gs.predict(X_test_new)  # predictions on test 
    
print classification_report(y_test,y_pred)

Fitting 5 folds for each of 70 candidates, totalling 350 fits


[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:    2.4s


             precision    recall  f1-score   support

          0       0.91      0.99      0.95       196
          1       0.50      0.05      0.09        21

avg / total       0.87      0.90      0.87       217



[Parallel(n_jobs=-1)]: Done 350 out of 350 | elapsed:    5.5s finished


In [41]:
rfc_gs.score(X_test_new,y_test)

0.90322580645161288

In [42]:
importance = rfc_gs.best_estimator_.feature_importances_

In [43]:
pd.DataFrame(importance,columns=['features_importances'],index=X_test_new.columns)\
.sort_values('features_importances',ascending=False).head(10)

,features_importances
experience,0.147521
working,0.116993
research,0.091700
team,0.088136
analysis,0.074727
knowledge,0.061104
management,0.055754
engineering,0.041930
work,0.041785
business,0.040578


## What components of a job posting distinguish data scientists from other data jobs?

In [103]:
# Isolate to just the portion of requirements/skill sets
df_new = df[df['is_duplicated'] == False]

In [104]:
df_new.loc[:,"all info"] = df_new["all info"].str.lower()

In [105]:
def extract_skills(df):
    
    skills = []
    for i,row in df.iterrows():
        try:
            df.loc[i,"experience"] = [x.strip() for x in re.findall('experience(.+)', str(df.loc[i,'all info']), re.MULTILINE | re.DOTALL)]
        except:
            df.loc[i,"experience"] = ""

    return df

In [106]:
df_new_exp = extract_skills(df_new)

In [107]:
df_new_exp

,all info,company name,country,job title,link,salary,has salary,is_duplicated,experience
0,the purpose of the role is to provide advanced...,Options Group,Singapore,"<font size=""+1"">Data Scientist / Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[/ skills -ba or ma degree in technical discip...
1,the big data engineer implements and maintains...,Siemens AG,Singapore,"<font size=""+1"">Big Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[experience with modern big data technologies ...
2,"incorporated in 1984, main board-listed singap...",Singapore Press Holdings,North Singapore,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[.the incumbent is expected to:· help to under...
3,you are responsible formanages the early invol...,Philips,Singapore,"<font size=""+1"">Supplier Quality Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[bachelor's degree in mechanical engineering o...
4,"in this intern program, you will be part of ou...",PropertyGuru,Singapore,"<font size=""+1"">Data engineer intern</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[to build some important features of our data ...
5,you are interested in building robust data pla...,Vault Dragon,Toa Payoh,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,
6,do you have a passion for data? are you lookin...,Moka,Singapore,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,"[) in building automated, scalable and robust ..."
7,responsibilities:conduct research and particip...,DSO National Laboratories,Singapore,"<font size=""+1"">Data Analytics Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[in machine learning will be advantageous]
8,healint is a leading maker of healthcare techn...,Healint,Pasir Panjang,"<font size=""+1"">Data Engineer</font>",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,[design to empower people to manage their chro...
9,"data visualization engineer,-174900description...",Teradata,Singapore,"<font size=""+1"">Data Visualization Engineer,</...",h,"<div data-tn-component=""jobHeader"">\n<b class=...",0,False,"[(i.e. visualising complex data, designing dat..."


### Re-create model to check job category (data scientist vs not data scientist)

In [108]:
df_new_exp.loc[:,"has scientist"] = 0

df_new_exp.loc[(df_new_exp["job title"].str.contains("Scientist") |
        df_new_exp["job title"].str.contains("Scientist")), "has scientist"] = 1

In [109]:
df_new_exp = df_new_exp.dropna(subset=['experience'])

In [110]:
df_new_exp.loc[:,"experience"] = df_new_exp["experience"].map(remove_lists)

In [111]:
df_new_exp = df_new_exp.dropna(subset=['experience'])

In [112]:
for i in df_new_exp["experience"]:
    print i
    print '------------------'

/ skills -ba or ma degree in technical disciplines including engineering, computer science, and applied sciences. background in fin. eng., math, or stats is also desirable. -experience working with large unstructured data sets and distributed computing tools (mapreduce, spark, hadoop, hive, etc.) -experience with large enterprise databases, in traditional/alternative database technologies (sql and nosql) -experience in statistical modelling and machine learning to develop robust predictive models -ability to quickly prototype solutions for diverse needs -ability to scrape and retrieve data from a variety of unstructured sources (web, text) -data manipulation –comfortable handling data types across varying frequencies and complexities -fluency in r, python and/or other similar languages -adept in the use of third-party visualization tools (spotfire, tableau) -ability to work in a challenging environment and creative in coming up with solutions to solve complex problems
-----------------

· uncover the actionable insights for multiple stakeholders to drive business growththe mission of data team is to drive sp to become data-driven company and create data-driven products. as a data team member, you will be responsible for designing, developing and deploying data-driven solutions to create business value. we are looking for a data scientist who will help us discover patterns hidden in large amounts of data and make decisions from different sources. your primary focus will be in applying data wrangling and machine learning techniques to build high quality anomaly detection, prediction and recommendation systems integrated with our products. you will work closely with customers and data engineers to understand the business requirements, in-house infrastructure and help build solutions for different business users.responsibilities:</b>· understand business logics from domain experts and come up with reasonable targets for data projects· data fetching from different sources 

In [113]:
X_exp = df_new_exp[["experience"]]
y_exp = df_new_exp["has scientist"]

In [114]:
y_exp.value_counts()

0    539
1     63
Name: has scientist, dtype: int64

In [115]:
X_exp.shape

(602, 1)

In [116]:
X_train_exp, X_test_exp, y_train_exp, y_test_exp = train_test_split(X_exp, y_exp, test_size=0.33, random_state=2,stratify=y_exp)

In [117]:
X_train_exp.reset_index(drop=True, inplace=True)
X_test_exp.reset_index(drop=True, inplace=True)
y_train_exp.reset_index(drop=True, inplace=True)
y_test_exp.reset_index(drop=True, inplace=True)

In [118]:
X_train_exp_new = vectorizer_fitting(TfidfVectorizer, X_train_exp, X_test_exp, stop_words, train=True, field="experience")
X_test_exp_new = vectorizer_fitting(TfidfVectorizer, X_train_exp, X_test_exp, stop_words, train=False, field="experience")

In [119]:
X_train_exp_new.head()

,ability,analysis,business,communication,development,experience,good,knowledge,management,process,skills,strong,support,team,technical,tools,understanding,work,working,years
0,0.000000,0.000000,0.000000,0.00000,0.00000,0.235151,0.795976,0.000000,0.363851,0.000000,0.397832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143071,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.21824,0.00000,0.161043,0.272562,0.416890,0.498366,0.000000,0.544910,0.255539,0.000000,0.000000,0.000000,0.000000,0.260297,0.000000,0.000000,0.000000
2,0.139907,0.000000,0.536932,0.00000,0.00000,0.098744,0.000000,0.000000,0.000000,0.174044,0.222742,0.313368,0.380503,0.265146,0.331626,0.000000,0.319203,0.240312,0.000000,0.139907
3,0.000000,0.154698,0.385955,0.00000,0.00000,0.094638,0.160173,0.122494,0.146434,0.000000,0.106740,0.000000,0.182341,0.508242,0.476756,0.160809,0.152965,0.115160,0.381182,0.134090
4,0.000000,0.000000,0.000000,0.00000,0.46582,0.276317,0.000000,0.357648,0.000000,0.000000,0.311651,0.000000,0.000000,0.000000,0.463997,0.000000,0.000000,0.336234,0.000000,0.391504


In [120]:
lr = LogisticRegression(random_state=123)
cross_val_score(lr,X_train_exp_new,y_train_exp)

array([ 0.8962963 ,  0.89552239,  0.89552239])

In [121]:
def above_below_comparision(df, field):
    
    #creating separate lists of the top appearing words so that these can be combined and compared to create ratios of appearances in above-median vs. below-median salaries

    word_count_below = df[df[field]==0].sum(axis=0)

    word_count_above = df[df[field]==1].sum(axis=0)

    #creating as dataframe, using method ".T" to transpose columns with index.

    word_count_compare = pd.DataFrame([word_count_below, word_count_above]).T

    #creating a "ratio" column to determine frequency of words associated with above median jobs vs. below median jobs

    word_count_compare["above_below_ratio"] = word_count_compare[1]/word_count_compare[0]

    return word_count_compare.above_below_ratio.sort_values(ascending=False)

In [122]:
X_train_exp_new

,ability,analysis,business,communication,development,experience,good,knowledge,management,process,skills,strong,support,team,technical,tools,understanding,work,working,years
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.235151,0.795976,0.000000,0.363851,0.000000,0.397832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143071,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.218240,0.000000,0.161043,0.272562,0.416890,0.498366,0.000000,0.544910,0.255539,0.000000,0.000000,0.000000,0.000000,0.260297,0.000000,0.000000,0.000000
2,0.139907,0.000000,0.536932,0.000000,0.000000,0.098744,0.000000,0.000000,0.000000,0.174044,0.222742,0.313368,0.380503,0.265146,0.331626,0.000000,0.319203,0.240312,0.000000,0.139907
3,0.000000,0.154698,0.385955,0.000000,0.000000,0.094638,0.160173,0.122494,0.146434,0.000000,0.106740,0.000000,0.182341,0.508242,0.476756,0.160809,0.152965,0.115160,0.381182,0.134090
4,0.000000,0.000000,0.000000,0.000000,0.465820,0.276317,0.000000,0.357648,0.000000,0.000000,0.311651,0.000000,0.000000,0.000000,0.463997,0.000000,0.000000,0.336234,0.000000,0.391504
5,0.301859,0.000000,0.289617,0.288714,0.000000,0.213047,0.360577,0.275755,0.000000,0.000000,0.240290,0.000000,0.000000,0.286035,0.000000,0.000000,0.000000,0.518489,0.286035,0.000000
6,0.158552,0.365841,0.000000,0.000000,0.000000,0.335710,0.568182,0.434524,0.346298,0.197239,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.180871,0.000000,0.000000,0.158552
7,0.135761,0.000000,0.130255,0.000000,0.000000,0.574907,0.324339,0.248042,0.296519,0.168887,0.000000,0.000000,0.000000,0.000000,0.000000,0.325626,0.309744,0.000000,0.385933,0.000000
8,0.000000,0.000000,0.132042,0.263260,0.000000,0.582792,0.328787,0.000000,0.150293,0.171203,0.547763,0.000000,0.000000,0.000000,0.163106,0.000000,0.000000,0.000000,0.260817,0.137623
9,0.219489,0.000000,0.421175,0.209931,0.000000,0.154911,0.000000,0.000000,0.000000,0.000000,0.349441,0.000000,0.000000,0.207983,0.000000,0.000000,0.000000,0.565508,0.415966,0.219489


In [123]:
df_y_train_exp = pd.DataFrame(y_train_exp)

In [124]:
job_exp = pd.concat([X_train_exp_new, df_y_train_exp], axis=1)

In [125]:
job_exp["has scientist"].value_counts()

0    361
1     42
Name: has scientist, dtype: int64

In [126]:
above_below_comparision(job_exp, "has scientist")

has scientist         inf
working          0.222733
knowledge        0.209184
technical        0.192507
tools            0.183026
understanding    0.166260
analysis         0.156251
experience       0.150197
good             0.148459
team             0.114720
years            0.107828
business         0.103983
strong           0.098684
development      0.097918
skills           0.074578
work             0.055510
ability          0.055127
management       0.044222
communication    0.043321
process          0.036138
support          0.003813
Name: above_below_ratio, dtype: float64

## Pick keywords 

In [127]:
keywords = ['machine learning', 'big data', 'data mining', 'deep learning', 'python', 'predictive modeling', \
            'clustering', "mapreduce", "spark", "hadoop", "hive"]

In [128]:
def create_columns(df, keywords):
    
    for i in keywords:
        df.loc[:, i] = 0
        df.loc[(df["experience"].str.contains(i)), i] = 1
        #df = df.drop('experience', axis=1)
        #df = df.astype("float")
        
    return df

In [136]:
X_skills = create_columns(X_exp, keywords)

In [137]:
X_skills = X_skills.drop('experience', axis=1)
X_skills

,machine learning,big data,data mining,deep learning,python,predictive modeling,clustering,mapreduce,spark,hadoop,hive
0,1,0,0,0,1,0,0,1,1,1,1
1,0,1,0,0,1,0,0,1,1,1,1
2,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,1,0,0,1,1,0,0
7,1,0,0,0,0,0,0,0,0,0,0
8,1,1,0,0,1,0,1,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0


In [138]:
X_train_skills, X_test_skills, y_train_skills, y_test_skills = train_test_split(X_skills, y_exp, test_size=0.33, random_state=2,stratify=y_exp)

In [131]:
job_skills = pd.concat([X_skills, y_exp], axis=1)

In [142]:
lr = LogisticRegression(random_state=12)
cross_val_score(lr,X_train_skills,y_train_skills)

array([ 0.91851852,  0.91791045,  0.92537313])

In [143]:
lr.fit(X_train_skills, y_train_skills)
lr.score(X_test_skills, y_test_skills)   # Accuracy on test set

0.92964824120603018